### 1. Load packages

In [1]:
import time

In [3]:
!pip install tinkoff-investments

In [5]:
TOKEN = '' #боевой M13

In [7]:
from tinkoff.invest import Client
#from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX 


# with Client(TOKEN, target=INVEST_GRPC_API_SANDBOX ) as client:
#     print(client.users.get_accounts())


with Client(TOKEN) as client:
    print(client.users.get_accounts())

GetAccountsResponse(accounts=[Account(id='2169433725', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='M13', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2022, 4, 27, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_READ_ONLY: 2>)])


### 2. Load data

In [9]:
import logging
import os

import pandas as pd

from tinkoff.invest import Client, SecurityTradingStatus
from tinkoff.invest.services import InstrumentsService
from tinkoff.invest.utils import quotation_to_decimal

from IPython.display import clear_output

In [17]:
def get_figi(ticker):
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        tickers = []
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                tickers.append(
                    {
                        "name": item.name,
                        "ticker": item.ticker,
                        "class_code": item.class_code,
                        "figi": item.figi,
                        "uid": item.uid,
                        "type": method,
                        "min_price_increment": quotation_to_decimal(
                            item.min_price_increment
                        ),
                        "scale": 9 - len(str(item.min_price_increment.nano)) + 1,
                        "lot": item.lot,
                        "trading_status": str(
                            SecurityTradingStatus(item.trading_status).name
                        ),
                        "api_trade_available_flag": item.api_trade_available_flag,
                        "currency": item.currency,
                        "exchange": item.exchange,
                        "buy_available_flag": item.buy_available_flag,
                        "sell_available_flag": item.sell_available_flag,
                        "short_enabled_flag": item.short_enabled_flag,
                        "klong": quotation_to_decimal(item.klong),
                        "kshort": quotation_to_decimal(item.kshort),
                    }
                )

        tickers_df = pd.DataFrame(tickers)

        ticker_df = tickers_df[tickers_df["ticker"] == ticker]
        #return ticker_df
        if ticker_df.empty:
            print("There is no such ticker: %s", ticker)
            return

        figi = ticker_df["figi"].iloc[0]
        return figi


def get_info():
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        tickers = []
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                tickers.append(
                    {
                        "name": item.name,
                        "ticker": item.ticker,
                        "class_code": item.class_code,
                        "figi": item.figi,
                        "uid": item.uid,
                        "type": method,
                        "min_price_increment": quotation_to_decimal(
                            item.min_price_increment
                        ),
                        "scale": 9 - len(str(item.min_price_increment.nano)) + 1,
                        "lot": item.lot,
                        "trading_status": str(
                            SecurityTradingStatus(item.trading_status).name
                        ),
                        "api_trade_available_flag": item.api_trade_available_flag,
                        "currency": item.currency,
                        "exchange": item.exchange,
                        "buy_available_flag": item.buy_available_flag,
                        "sell_available_flag": item.sell_available_flag,
                        "short_enabled_flag": item.short_enabled_flag,
                        "klong": quotation_to_decimal(item.klong),
                        "kshort": quotation_to_decimal(item.kshort),
                    }
                )

        tickers_df = pd.DataFrame(tickers)
    return tickers_df

In [11]:
import os
from datetime import timedelta
import datetime

from tqdm import tqdm
import numpy as np

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

    
def get_all_candles(ticker=None, from_=None, to_=None, interval=None):
    
    figi = get_figi(ticker)
    
    with Client(TOKEN) as client:
        
        data = [elem for elem in client.get_all_candles(figi=figi, 
                                      from_=from_-timedelta(hours=3),
                                      to=to_- timedelta(hours=3),
                                     interval=interval
                                    )
               ]

    
    #open_ = np.zeros(len(data))
    #high = np.zeros(len(data))
    #low = np.zeros(len(data))
    close = np.zeros(len(data))
    volume = np.zeros(len(data))
    times = np.zeros(len(data), dtype='O')
    for i, elem in enumerate(data):
        #open_[i] = quotation_to_decimal(elem.open)
        #high[i] = quotation_to_decimal(elem.high)
        #low[i] = quotation_to_decimal(elem.low)
        close[i] = quotation_to_decimal(elem.close)
        volume[i] = elem.volume
        times[i] = elem.time


    df = pd.DataFrame({'time' : times,
                      #'open' : open_,
                      'close' : close,
                      'volume' : volume,
                      #'low' : low,
                      #'high' : high
                    })
    df['time'] += pd.Timedelta(hours=3)
    return df

### -1


In [77]:
get_figi('SFIN')

'BBG003LYCMB1'

In [79]:
ticker = 'T'
#2024-06-10 
data = get_all_candles(ticker=ticker,
                       from_=datetime.datetime(2024, 6, 10, tzinfo=datetime.timezone.utc),
                       to_=datetime.datetime(2024, 6, 12, tzinfo=datetime.timezone.utc),
                       interval=CandleInterval.CANDLE_INTERVAL_HOUR
                      )

data['ticker'] = ticker
data

,time,close,volume,ticker
0,2024-06-10 09:00:00+00:00,2974.0,1846.0,T
1,2024-06-10 10:00:00+00:00,2989.0,252336.0,T
2,2024-06-10 11:00:00+00:00,2976.0,106799.0,T
3,2024-06-10 12:00:00+00:00,2977.6,139763.0,T
4,2024-06-10 13:00:00+00:00,2973.6,64276.0,T
5,2024-06-10 14:00:00+00:00,2974.0,66350.0,T
6,2024-06-10 15:00:00+00:00,2948.6,133118.0,T
7,2024-06-10 16:00:00+00:00,2947.0,69133.0,T
8,2024-06-10 17:00:00+00:00,2930.6,86918.0,T
9,2024-06-10 18:00:00+00:00,2898.0,161966.0,T


In [81]:
data = get_all_candles(ticker=ticker,
                       from_=datetime.datetime(2024, 6, 10, tzinfo=datetime.timezone.utc),
                       to_=datetime.datetime(2024, 6, 12, tzinfo=datetime.timezone.utc),
                       interval=CandleInterval.CANDLE_INTERVAL_DAY
                      )

data['ticker'] = ticker
data

,time,close,volume,ticker
0,2024-06-10 03:00:00+00:00,2886.6,1391052.0,T
1,2024-06-11 03:00:00+00:00,2890.6,1160771.0,T


### -1

In [86]:
%%time

ticker = 'TMOS'

data = get_all_candles(ticker=ticker,
                       from_=datetime.datetime(2025, 1, 1, tzinfo=datetime.timezone.utc),
                       to_=datetime.datetime(2025, 1, 10, tzinfo=datetime.timezone.utc),
                       interval=CandleInterval.CANDLE_INTERVAL_HOUR
                      )

data['ticker'] = ticker

CPU times: user 719 ms, sys: 7.36 ms, total: 727 ms
Wall time: 1.22 s


In [87]:
data

,time,close,volume,ticker
0,2025-01-03 09:00:00+00:00,6.31,81165.0,TMOS
1,2025-01-03 10:00:00+00:00,6.20,12405231.0,TMOS
2,2025-01-03 11:00:00+00:00,6.19,4224743.0,TMOS
3,2025-01-03 12:00:00+00:00,6.14,5029243.0,TMOS
4,2025-01-03 13:00:00+00:00,6.14,4735730.0,TMOS
5,2025-01-03 14:00:00+00:00,6.14,6350496.0,TMOS
6,2025-01-03 15:00:00+00:00,6.14,1389740.0,TMOS
7,2025-01-03 16:00:00+00:00,6.14,2190425.0,TMOS
8,2025-01-03 17:00:00+00:00,6.15,2098042.0,TMOS
9,2025-01-03 18:00:00+00:00,6.15,695071.0,TMOS


In [24]:
!mkdir data

In [26]:
!mkdir data/raw

In [28]:
!mkdir data/raw/1min

In [13]:
import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data

### Отберем ликвидные акции

In [37]:
stocks = ['AFLT',
 'IRAO',
 'MVID',
 'OGKB',
 'SPBE',
 'MTLRP',
 'SFIN',
 'KMAZ',
 'CBOM',
 'MSNG',
 'FEES',
 'LKOH',
 'FESH',
 'KROT',
 'LENT',
 'MDMG',
 'NKNCP',
 'VKCO',
 'UPRO',
 'MRKY',
 'SELG',
 'SGZH',
 'AKRN',
 'CHMK',
 'ENPG',
 'YAKG',
 'PMSBP',
 'RKKE',
 'GEMC',
 'LSRG',
 'MRKZ',
 'BANE',
 'CNTL',
 'PMSB',
 'KRKNP',
 'NMTP',
 'PLZL',
 'TATN',
 'AFKS',
 'UNKL',
 'APTK',
 'LNZL',
 'GMKN',
 'NSVZ',
 'LIFE',
 'RNFT',
 'VRSB',
 'YDEX',
 'UWGN',
 'TGKN',
 'ABRD',
 'HYDR',
 'ABIO',
 'WUSH',
 'RTKMP',
 'GTRK',
 'NLMK',
 'TATNP',
 'CNTLP',
 'BLNG',
 'ALRS',
 'GCHE',
 'VSMO',
 'LSNG',
 'TRNFP',
 'MGNT',
 'RENI',
 'NKHP',
 'TGKB',
 'ROLO',
 'IRKT',
 'MGTSP',
 'MTLR',
 'TGKA',
 'MSRS',
 'HEAD',
 'RUAL',
 'MRKV',
 'KLSB',
 'SIBN',
 'SNGS',
 'MSTT',
 'KAZTP',
 'DVEC',
 'NVTK',
 'RASP',
 'VTBR',
 'PHOR',
 'TTLK',
 'NKNC',
 'T',
 'KAZT',
 'AMEZ',
 'MRKU',
 'TRMK',
 'MOEX',
 'SBERP',
 #'VEON-RX',
 'CHMF',
 'RTKM',
 'SMLT',
 'LNZLP',
 'MRKC',
 'MRKP',
 'UNAC',
 'KZOSP',
 'GAZP',
 'ROSN',
 'BELU',
 'ELFV',
 'FLOT',
 'PIKK',
 'LSNGP',
 'SVAV',
 'TGKBP',
 'POSI',
 'KZOS',
 'MRKS',
 'SNGSP',
 'MTSS',
 'MAGN',
 'PRFN',
 'SBER',
 'BANEP',
 'BSPB',
 'AQUA',
 'RBCM']

In [112]:
data = []
for stock in stocks:
    df = load_pkl(f"../money_v3/data/raw/1day/{stock}.pkl")
    df = df[df['time'] > datetime.datetime(2025, 1, 1, tzinfo=datetime.timezone.utc)]
    data += [df]
data = pd.concat(data)

data = data.groupby('ticker').apply(lambda x: pd.Series({'mean_volume' :  x['volume'].mean(),
                                                         'mean_close' : x['close'].mean()}))
data

/var/folders/rm/zt5_90s5727289mv2gzbkw980000gn/T/ipykernel_75878/1407591603.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('ticker').apply(lambda x: pd.Series({'mean_volume' :  x['volume'].mean(),


,mean_volume,mean_close
ticker,,
ABIO,1.996380e+05,90.305455
ABRD,1.130664e+04,186.772727
AFKS,1.342772e+06,14.786773
AFLT,3.610632e+06,61.276818
AKRN,4.325909e+02,17686.636364
ALRS,2.243179e+06,56.313636
AMEZ,6.639273e+03,71.109091
APTK,5.555096e+05,11.892182
AQUA,9.437355e+04,634.681818


In [114]:
data_info = get_info()
data_info

,name,ticker,class_code,figi,uid,type,min_price_increment,scale,lot,trading_status,api_trade_available_flag,currency,exchange,buy_available_flag,sell_available_flag,short_enabled_flag,klong,kshort
0,CNX Resources Corporation,CNX,SPBXM,BBG000CKVSG8,82f8d11f-0307-45c3-bb94-8758a2f673a3,shares,0.01000000,2,1,SECURITY_TRADING_STATUS_BREAK_IN_TRADING,True,usd,unknown,False,False,False,0E-8,0E-8
1,SLM Corp,SLM,SPBXM,BBG000BBCQD7,f8e85601-10e3-4af6-a44b-b7d2ca3f7f62,shares,0.01000000,2,1,SECURITY_TRADING_STATUS_BREAK_IN_TRADING,True,usd,unknown,False,False,False,0E-8,0E-8
2,Axcelis Technologies,ACLS,SPBXM,BBG000DW34S2,98da7119-dc33-4567-ae41-1944f6f49334,shares,0.01000000,2,1,SECURITY_TRADING_STATUS_BREAK_IN_TRADING,True,usd,unknown,False,False,False,0E-8,0E-8
3,Amphastar Pharmaceuticals,AMPH,SPBXM,BBG000BW90S6,c1ad8284-45de-468f-8a37-271f254ae601,shares,0.01000000,2,1,SECURITY_TRADING_STATUS_BREAK_IN_TRADING,True,usd,unknown,False,False,False,0E-8,0E-8
4,Remitly Global,RELY,SPBXM,BBG012C7JST2,fb4f9c78-ba9a-4733-be60-50d6eacea5e0,shares,0.01000000,2,1,SECURITY_TRADING_STATUS_BREAK_IN_TRADING,True,usd,unknown,False,False,False,0E-8,0E-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3828,BANE-6.25 Башнефть,BNM5,SPBFUT,FUTBANE06250,a4811010-1dcf-4bb3-ac83-4c83a507834e,futures,1.00000000,9,1,SECURITY_TRADING_STATUS_NORMAL_TRADING,True,rub,FORTS_EVENING,True,True,True,2.00000000,2.00000000
3829,MMI-6.25 Индекс Металлов и добычи,MAM5,SPBFUT,FUTMMI062500,742d7110-045f-4166-b24f-6019b385eeba,futures,1.00000000,9,1,SECURITY_TRADING_STATUS_NORMAL_TRADING,True,rub,FORTS_EVENING,True,True,True,2.00000000,2.00000000
3830,RTSM-9.25 Индекс РТС (мини),RMU5,SPBFUT,FUTRTSM09250,106e6bcb-627d-4ad7-8e96-141527bbdbd1,futures,0.50000000,1,1,SECURITY_TRADING_STATUS_NORMAL_TRADING,True,rub,FORTS_EVENING,True,True,True,2.00000000,2.00000000
3831,BR-5.25 Нефть Brent,BRK5,SPBFUT,FUTBR0525000,162b7aab-c5b1-45d2-9353-363dad9510c1,futures,0.01000000,2,1,SECURITY_TRADING_STATUS_NORMAL_TRADING,True,rub,FORTS_EVENING,True,True,True,2.00000000,2.00000000


In [117]:
data = data.merge(data_info[['ticker', 'lot']], on='ticker', how='left')
data

,ticker,mean_volume,mean_close,lot
0,ABIO,1.996380e+05,90.305455,10
1,ABRD,1.130664e+04,186.772727,10
2,AFKS,1.342772e+06,14.786773,100
3,AFLT,3.610632e+06,61.276818,10
4,AKRN,4.325909e+02,17686.636364,1
5,ALRS,2.243179e+06,56.313636,10
6,AMEZ,6.639273e+03,71.109091,100
7,APTK,5.555096e+05,11.892182,10
8,AQUA,9.437355e+04,634.681818,1
9,BANE,3.183355e+04,2494.431818,1


In [119]:
data['ticker'].value_counts()

ticker
T        2
ABIO     1
SBER     1
RTKMP    1
RTKM     1
ROSN     1
ROLO     1
RNFT     1
RKKE     1
RENI     1
RBCM     1
RASP     1
PRFN     1
POSI     1
PMSBP    1
PMSB     1
PLZL     1
PIKK     1
PHOR     1
OGKB     1
NVTK     1
NSVZ     1
NMTP     1
NLMK     1
NKNCP    1
NKNC     1
NKHP     1
MVID     1
MTSS     1
MTLRP    1
MTLR     1
RUAL     1
SBERP    1
ABRD     1
SELG     1
YAKG     1
WUSH     1
VTBR     1
VSMO     1
VRSB     1
VKCO     1
UWGN     1
UPRO     1
UNKL     1
UNAC     1
TTLK     1
TRNFP    1
TRMK     1
TGKN     1
TGKBP    1
TGKB     1
TGKA     1
TATNP    1
TATN     1
SVAV     1
SPBE     1
SNGSP    1
SNGS     1
SMLT     1
SIBN     1
SGZH     1
SFIN     1
MSTT     1
MSRS     1
MSNG     1
CHMK     1
GTRK     1
GMKN     1
GEMC     1
GCHE     1
GAZP     1
FLOT     1
FESH     1
FEES     1
ENPG     1
ELFV     1
DVEC     1
CNTLP    1
CNTL     1
CHMF     1
MRKZ     1
CBOM     1
BSPB     1
BLNG     1
BELU     1
BANEP    1
BANE     1
AQUA     1
APTK     1
AMEZ     1
ALR

In [122]:
data['liquid_cap'] = data['mean_volume']*data['mean_close']*data['lot'] / 1e6
data

,ticker,mean_volume,mean_close,lot,liquid_cap
0,ABIO,1.996380e+05,90.305455,10,180.283962
1,ABRD,1.130664e+04,186.772727,10,21.117713
2,AFKS,1.342772e+06,14.786773,100,1985.526372
3,AFLT,3.610632e+06,61.276818,10,2212.480267
4,AKRN,4.325909e+02,17686.636364,1,7.651078
5,ALRS,2.243179e+06,56.313636,10,1263.215819
6,AMEZ,6.639273e+03,71.109091,100,47.211265
7,APTK,5.555096e+05,11.892182,10,66.062211
8,AQUA,9.437355e+04,634.681818,1,59.897173
9,BANE,3.183355e+04,2494.431818,1,79.406609


In [124]:
pd.set_option('display.max_rows', 130)
data.sort_values(by='liquid_cap')

,ticker,mean_volume,mean_close,lot,liquid_cap
35,KAZTP,3.508182e+02,502.295455,10,1.762144
38,KRKNP,2.972727e+02,8138.181818,1,2.419260
111,TGKBP,2.645591e+03,0.012511,100000,3.309995
81,PMSBP,1.288591e+03,280.963636,10,3.620472
18,CNTLP,5.573682e+03,7.618182,100,4.246132
34,KAZT,8.894545e+02,500.509091,10,4.451801
41,KZOSP,2.155391e+04,21.179091,10,4.564922
46,LNZLP,2.721591e+03,1785.227273,1,4.858658
16,CHMK,7.711818e+02,7128.863636,1,5.497650
53,MGTSP,7.077136e+03,871.818182,1,6.169976


In [130]:
(data['liquid_cap'] > 50).sum() - 1 # 100млн руб

76

In [132]:
data.loc[data['liquid_cap'] > 50, 'ticker'].tolist()

['ABIO',
 'AFKS',
 'AFLT',
 'ALRS',
 'APTK',
 'AQUA',
 'BANE',
 'BANEP',
 'BELU',
 'BSPB',
 'CBOM',
 'CHMF',
 'ENPG',
 'FEES',
 'FESH',
 'FLOT',
 'GAZP',
 'GMKN',
 'GTRK',
 'HEAD',
 'HYDR',
 'IRAO',
 'IRKT',
 'KMAZ',
 'LENT',
 'LIFE',
 'LKOH',
 'LSRG',
 'MAGN',
 'MDMG',
 'MGNT',
 'MOEX',
 'MTLR',
 'MTLRP',
 'MTSS',
 'MVID',
 'NLMK',
 'NMTP',
 'NVTK',
 'OGKB',
 'PHOR',
 'PIKK',
 'PLZL',
 'POSI',
 'RASP',
 'RENI',
 'RNFT',
 'ROSN',
 'RTKM',
 'RTKMP',
 'RUAL',
 'SBER',
 'SBERP',
 'SELG',
 'SFIN',
 'SGZH',
 'SIBN',
 'SMLT',
 'SNGS',
 'SNGSP',
 'SPBE',
 'SVAV',
 'T',
 'T',
 'TATN',
 'TATNP',
 'TGKN',
 'TRMK',
 'TRNFP',
 'UNAC',
 'UPRO',
 'UWGN',
 'VKCO',
 'VSMO',
 'VTBR',
 'WUSH',
 'YDEX']

# 1. Get data

In [15]:
stocks = ['ABIO',
 'AFKS',
 'AFLT',
 'ALRS',
 'APTK',
 'AQUA',
 'BANE',
 'BANEP',
 'BELU',
 'BSPB',
 'CBOM',
 'CHMF',
 'ENPG',
 'FEES',
 'FESH',
 'FLOT',
 'GAZP',
 'GMKN',
 'GTRK',
 'HEAD',
 'HYDR',
 'IRAO',
 'IRKT',
 'KMAZ',
 'LENT',
 'LIFE',
 'LKOH',
 'LSRG',
 'MAGN',
 'MDMG',
 'MGNT',
 'MOEX',
 'MTLR',
 'MTLRP',
 'MTSS',
 'MVID',
 'NLMK',
 'NMTP',
 'NVTK',
 'OGKB',
 'PHOR',
 'PIKK',
 'PLZL',
 'POSI',
 'RASP',
 'RENI',
 'RNFT',
 'ROSN',
 'RTKM',
 'RTKMP',
 'RUAL',
 'SBER',
 'SBERP',
 'SELG',
 'SFIN',
 'SGZH',
 'SIBN',
 'SMLT',
 'SNGS',
 'SNGSP',
 'SPBE',
 'SVAV',
 'T',
 'TATN',
 'TATNP',
 'TGKN',
 'TRMK',
 'TRNFP',
 'UNAC',
 'UPRO',
 'UWGN',
 'VKCO',
 'VSMO',
 'VTBR',
 'WUSH',
 'YDEX',


'TMOS',
]

start = datetime.datetime(2023, 7, 1, tzinfo=datetime.timezone.utc)
end = datetime.datetime(2025, 2, 25, tzinfo=datetime.timezone.utc)

In [17]:
len(stocks) #stocks + TMOS

77

In [102]:
#1min

#проблемы со сборкой данных на стороне сервера Т-банка
for i, stock in tqdm(enumerate(stocks)):
    
    dfs = []
    cur_date = start
    while cur_date < end:
    
        flag_data = False
        while not flag_data:
            
            clear_output()
            print(i, 'of', len(stocks), stock, cur_date)
            try:
                
                ########
                while (pd.Timestamp.now().minute % 15 < 2) or (pd.Timestamp.now().minute == 10): #для параллельной работы прома
                    time.sleep(5)
                ########
                
                data = get_all_candles(ticker=stock,
                               from_=cur_date,
                               to_=cur_date+datetime.timedelta(days=30),
                               interval=CandleInterval.CANDLE_INTERVAL_1_MIN
                              )
                dfs += [data.copy()]
                flag_data = True
                cur_date = cur_date+datetime.timedelta(days=30)
            except:
                print('Error: flag_data')
                time.sleep(5)
                
                

    data = pd.concat(dfs)
    data = data[data['time'] < end]
    data.sort_values(by='time', inplace=True)
    data.reset_index(drop=True, inplace=True)


    data['ticker'] = stock
    dump_pkl(data, f"./data/raw/1min/{stock}.pkl")

76 of 77 TMOS 2025-02-20 00:00:00+00:00


77it [2:37:44, 122.91s/it]


### 4. Preproc data

* Оставляем только будни
* И часы основных торгов - 9:55–23:45 (начало 5 минутной свечи)
* Добавим к акциям IMOEX

In [19]:
!mkdir data/preproc

mkdir: data/preproc: File exists


In [21]:
!mkdir data/preproc/1min

mkdir: data/preproc/1min: File exists


In [26]:
pd.set_option('display.max_rows', 20)

In [28]:
stock = "TMOS"
data_tmos = load_pkl(f"./data/raw/1min/{stock}.pkl")
data_tmos

,time,close,volume,ticker
0,2023-07-03 09:59:00+00:00,5.50,39557.0,TMOS
1,2023-07-03 10:00:00+00:00,5.51,50291.0,TMOS
2,2023-07-03 10:01:00+00:00,5.51,208282.0,TMOS
3,2023-07-03 10:02:00+00:00,5.51,107967.0,TMOS
4,2023-07-03 10:03:00+00:00,5.51,36087.0,TMOS
...,...,...,...,...
341584,2025-02-24 23:45:00+00:00,7.33,18019.0,TMOS
341585,2025-02-24 23:46:00+00:00,7.33,52095.0,TMOS
341586,2025-02-24 23:47:00+00:00,7.33,74623.0,TMOS
341587,2025-02-24 23:48:00+00:00,7.32,12659.0,TMOS


In [30]:
data_tmos.iloc[-12*14:]

,time,close,volume,ticker
341421,2025-02-24 21:02:00+00:00,7.27,2888.0,TMOS
341422,2025-02-24 21:03:00+00:00,7.26,32875.0,TMOS
341423,2025-02-24 21:04:00+00:00,7.27,31293.0,TMOS
341424,2025-02-24 21:05:00+00:00,7.27,5454.0,TMOS
341425,2025-02-24 21:06:00+00:00,7.27,681339.0,TMOS
...,...,...,...,...
341584,2025-02-24 23:45:00+00:00,7.33,18019.0,TMOS
341585,2025-02-24 23:46:00+00:00,7.33,52095.0,TMOS
341586,2025-02-24 23:47:00+00:00,7.33,74623.0,TMOS
341587,2025-02-24 23:48:00+00:00,7.32,12659.0,TMOS


In [32]:
#1 min
def is_business_day(date):
    return bool(len(pd.bdate_range(date, date)))

def preproc_1min(data):
    data['time'] = data['time'].apply(lambda x: x.tz_convert(None))
    
    mask_bd = np.array(data['time'].apply(lambda x: is_business_day(x)))
    data = data.loc[mask_bd, :]

    mask_volume = np.array(datetime.time(9, 59) <= pd.to_datetime(data['time'], format='%H:%M').dt.time)
    data = data.loc[mask_volume, :]

    data.reset_index(drop=True, inplace=True)

    return data


#1 min
for stock in tqdm(stocks):
    data = load_pkl(f"./data/raw/1min/{stock}.pkl")

    data_ext = data.merge(data_tmos[['time', 'close']].copy().rename(columns={"close" : "tmos_close"}), how='left', on='time')
    assert data.shape[0] == data_ext.shape[0], 'Error 1'

    data = preproc_1min(data_ext)
    print(stock, data['tmos_close'].isnull().sum())#праздники + спб биржа не останавливаент торги в 18:45
    data.loc[:, 'tmos_close'] = data['tmos_close'].ffill()
    
    dump_pkl(data, f"./data/preproc/1min/{stock}.pkl")



  1%|▉                                                                         | 1/77 [00:05<06:24,  5.05s/it]

ABIO 1666


  3%|█▉                                                                        | 2/77 [00:14<09:35,  7.68s/it]

AFKS 4204


  4%|██▉                                                                       | 3/77 [00:25<11:33,  9.37s/it]

...

WUSH 6066


 99%|████████████████████████████████████████████████████████████████████████ | 76/77 [10:49<00:09,  9.33s/it]

YDEX 7157


100%|█████████████████████████████████████████████████████████████████████████| 77/77 [10:57<00:00,  8.54s/it]

TMOS 0


In [ ]:
#Не так уж и много

In [44]:
load_pkl("./data/preproc/1min/YDEX.pkl").iloc[-60*14+30:]

,time,close,volume,ticker,tmos_close
320975,2025-02-21 23:49:00,4667.0,399.0,YDEX,7.21
320976,2025-02-24 09:59:00,4686.0,163.0,YDEX,7.24
320977,2025-02-24 10:00:00,4682.5,456.0,YDEX,7.26
320978,2025-02-24 10:01:00,4683.0,1356.0,YDEX,7.26
320979,2025-02-24 10:02:00,4684.5,1183.0,YDEX,7.26
...,...,...,...,...,...
321780,2025-02-24 23:45:00,4664.0,306.0,YDEX,7.33
321781,2025-02-24 23:46:00,4664.0,318.0,YDEX,7.33
321782,2025-02-24 23:47:00,4664.0,559.0,YDEX,7.33
321783,2025-02-24 23:48:00,4663.0,68.0,YDEX,7.32


#### 1 min

In [49]:
for stock in tqdm(stocks):
    df = load_pkl(f"./data/preproc/1min/{stock}.pkl")

    assert (df['time'].sort_values()  == df['time']).all(), f'{stock} not sorted: {(df['time'].sort_values()  != df['time']).sum()}'
    assert not df['time'].duplicated().any(), f'{stock} duplicated: {df['time'].duplicated().sum()}'
    assert df.notnull().all().all(), f'{stock} nulls'
    

100%|█████████████████████████████████████████████████████████████████████████| 77/77 [00:02<00:00, 31.27it/s]


In [51]:
#Нет дублей и данные отсортированы, нет null's (еще бы)

In [54]:
#Заполняемость

In [56]:


dfs = []
for stock in tqdm(stocks):
    df = load_pkl(f"./data/preproc/1min/{stock}.pkl")
    
    df['diff'] = df['time'].diff() / pd.Timedelta('1 min')
    dfs += [df[['diff']].copy()]
df = pd.concat(dfs)
df.shape

100%|█████████████████████████████████████████████████████████████████████████| 77/77 [00:01<00:00, 64.13it/s]


(22571457, 1)

In [58]:
df['diff'].quantile(q=[0, 0.01]+np.arange(0.1, 1, 0.1).tolist()+[0.99, 1])

0.00        1.0
0.01        1.0
0.10        1.0
0.20        1.0
0.30        1.0
0.40        1.0
0.50        1.0
0.60        1.0
0.70        1.0
0.80        1.0
0.90        1.0
0.99        4.0
1.00    68591.0
Name: diff, dtype: float64

In [60]:
#Норм

#### Удалим TMOS

In [64]:
import os
stock = "TMOS"

os.remove(f"./data/preproc/1min/{stock}.pkl")


### Поправим BELU после сплита

In [71]:
#1 min
stock = "BELU"

df = load_pkl(f"./data/preproc/1min/{stock}.pkl")

mask = df['time'] < pd.Timestamp('2024-08-22')
df.loc[mask, 'close'] = df.loc[mask, 'close'] / 8
df.loc[mask, 'volume'] = df.loc[mask, 'volume'] * 8

dump_pkl(df, f"./data/preproc/1min/{stock}.pkl")

# -2. Check data

### -2.1 VKCO

In [77]:
#чекнуть сами данные и совпадение с индексом

In [80]:
#1 min

In [82]:
i = -124
stock = "VKCO"

df = load_pkl(f"./data/preproc/1min/{stock}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,tmos_close
323087,2025-02-24 21:45:00,339.7,621.0,VKCO,7.29
323088,2025-02-24 21:46:00,340.3,25784.0,VKCO,7.29


### -2.2 YDEX

In [86]:
#1min
i = -1000

stock = "YDEX"
df = load_pkl(f"./data/preproc/1min/{stock}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,tmos_close
320784,2025-02-21 20:35:00,4658.5,358.0,YDEX,7.19
320785,2025-02-21 20:36:00,4658.0,1010.0,YDEX,7.18


### -2.3 SNGS

In [91]:
#1 min
i = -32

stock = "SNGS"
df = load_pkl(f"./data/preproc/1min/{stock}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,tmos_close
326850,2025-02-24 23:17:00,29.06,22.0,SNGS,7.3
326851,2025-02-24 23:18:00,29.06,775.0,SNGS,7.3
